In [1]:
# import libraries
import numpy as np
import pandas as pd
import Cleanup

In [2]:
# Call the clean() method from the Cleanup class to perform data cleanup or transformation
clean = Cleanup.clean()

In [3]:
df = pd.read_csv("merged.csv")
df.head()

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,...,Lane_ID_preceding,Preceding_preceding,Following_preceding,Space_Headway_preceding,Time_Headway_preceding,Location_preceding,Period_preceding,L-F_Pair,pair_Time_Duration,total_pair_duration
0,2,373,415,1.113430e+12,1.742846,29.798545,6042824.992,2133164.698,4.66344,6.4,...,1,3355,2,34.216848,43.51,i80,4:00PM - 4:15PM,44-2,0.0,10.7
1,1764,5707,712,1.113430e+12,8.642909,345.582333,6042711.299,2134196.335,4.66344,6.4,...,1,1758,1766,20.772120,1.33,i80,5:00PM - 5:15PM,1763-1764,0.0,79.4
2,1764,5707,712,1.113430e+12,8.642909,345.582333,6042711.299,2134196.335,4.66344,6.4,...,3,1756,1764,19.275552,1.78,i80,4:00PM - 4:15PM,1763-1764,0.1,79.4
3,1764,5706,712,1.113430e+12,8.643518,344.511617,6042711.847,2134192.895,4.66344,6.4,...,1,1758,1766,21.043392,1.35,i80,5:00PM - 5:15PM,1763-1764,0.2,79.4
4,1764,5706,712,1.113430e+12,8.643518,344.511617,6042711.847,2134192.895,4.66344,6.4,...,3,1756,1764,19.229832,1.90,i80,4:00PM - 4:15PM,1763-1764,0.3,79.4


In [4]:
df.columns

Index(['Vehicle_ID', 'Frame_ID', 'Total_Frames', 'Global_Time', 'Local_X',
       'Local_Y', 'Global_X', 'Global_Y', 'v_Length', 'v_Width', 'v_Class',
       'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding', 'Following', 'Space_Headway',
       'Time_Headway', 'Location', 'Period', 'Vehicle_ID_preceding',
       'Total_Frames_preceding', 'Global_Time_preceding', 'Local_X_preceding',
       'Local_Y_preceding', 'Global_X_preceding', 'Global_Y_preceding',
       'v_Length_preceding', 'v_Width_preceding', 'v_Class_preceding',
       'v_Vel_preceding', 'v_Acc_preceding', 'Lane_ID_preceding',
       'Preceding_preceding', 'Following_preceding', 'Space_Headway_preceding',
       'Time_Headway_preceding', 'Location_preceding', 'Period_preceding',
       'L-F_Pair', 'pair_Time_Duration', 'total_pair_duration'],
      dtype='object')

In [5]:
# Drop the specified columns from the DataFrame 'df'
df = df.drop(['Preceding_preceding','Following_preceding','Local_X_preceding', 'Local_Y_preceding',
              'Global_X_preceding', 'Global_Y_preceding','Total_Frames_preceding', 'Location_preceding', 'Period_preceding'],axis=1)

In [6]:
# So , for proper knowing the Vehicle type we are assigning the v_class with Vehicle class
 #   1 L Motorcycle   2: Car     3: Heavy Vehicle
    
filtered_vClass = df[['Vehicle_ID', 'v_Class', 'v_Length']].drop_duplicates().sort_values('v_Class')

class_mapping = {1: 'Motorcycle', 2: 'Car', 3: 'Heavy Vehicle'}

df['Vehicle_Class'] = df['v_Class'].map(class_mapping)


### Calculating the space gap by subtracting the length of the preceding vehicle from the space headway using the following code

In [7]:
# Calculate the space gap 

df['Space_Gap'] = df['Space_Headway'] - df['v_Length_preceding']

In [8]:
# Define the reaction times in seconds

df['Difference_of_speed'] = df['v_Vel_preceding'] - df['v_Vel']



In [9]:
# Filter the dataset to remove vehicles with time headway >= 5 seconds
df = df.groupby(['Vehicle_ID', 'Period']).filter(lambda x: (x['Time_Headway'] < 5).all()).reset_index(drop=True)


In [10]:
df

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_Length,v_Width,...,v_Acc_preceding,Lane_ID_preceding,Space_Headway_preceding,Time_Headway_preceding,L-F_Pair,pair_Time_Duration,total_pair_duration,Vehicle_Class,Space_Gap,Difference_of_speed
0,1760,5683,243,1.113430e+12,0.953414,425.375767,6042644.327,2134449.877,6.49224,8.5,...,-3.043688,1,33.634680,1.58,1754-1760,0.0,11.6,Heavy Vehicle,36.277296,1.090135
1,1760,5682,243,1.113430e+12,0.961339,423.323483,6042645.392,2134443.458,6.49224,8.5,...,-2.833217,1,34.192464,1.56,1754-1760,0.1,11.6,Heavy Vehicle,36.030408,1.327031
2,1760,5681,243,1.113430e+12,0.969264,421.265679,6042646.466,2134436.985,6.49224,8.5,...,-2.703060,1,34.475928,1.63,1754-1760,0.2,11.6,Heavy Vehicle,35.920680,1.527489
3,1760,5680,243,1.113430e+12,0.982370,419.201729,6042647.579,2134430.379,6.49224,8.5,...,-2.366147,1,34.847784,1.64,1754-1760,0.3,11.6,Heavy Vehicle,35.832288,1.700222
4,1760,5679,243,1.113430e+12,0.988771,417.131926,6042648.745,2134423.299,6.49224,8.5,...,-1.977164,1,35.295840,1.67,1754-1760,0.4,11.6,Heavy Vehicle,35.814000,1.844612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341701,1755,7322,388,1.113438e+12,0.874166,439.571071,6042636.659,2134495.641,5.42544,8.4,...,-0.428896,1,33.086040,2.39,1749-1755,32.4,32.8,Car,16.773144,1.164830
341702,1755,7323,388,1.113438e+12,0.869594,440.984301,6042635.877,2134500.351,5.42544,8.4,...,-0.702400,1,33.278064,2.43,1749-1755,32.5,32.8,Car,16.684752,1.071146
341703,1755,7324,388,1.113438e+12,0.865937,442.401238,6042635.103,2134505.014,5.42544,8.4,...,-0.960326,1,33.521904,2.44,1749-1755,32.6,32.8,Car,16.605504,0.951025
341704,1755,7325,388,1.113438e+12,0.861974,443.822373,6042634.346,2134509.574,5.42544,8.4,...,-1.217816,1,33.677352,2.41,1749-1755,32.7,32.8,Car,16.565880,0.795200


In [11]:
df.to_csv("Transformation_2.csv", index = False)